Author: Lourde Hajjar

This notebook applies a CNN to the edge-detected image dataset, initially testing a basic model without specifying a learning rate. The model was then enhanced by incorporating data augmentation (e.g., rotation, shifting, zooming, and flipping) to improve generalization and batch normalization layers to stabilize training.

The model’s performance was first evaluated using a test set and 10-fold cross-validation, achieving an accuracy of 63%. To further improve performance, a learning rate was added, resulting in an accuracy increase to 66%.

# CNN With Augmentation

## Load the Edge-Detected Dataset

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Define dataset path
data_dir =r"C:\Users\lourd\OneDrive\Desktop\coursework\datasets\3_image\processed\3_ed_b"


images = []
labels = []


# Load images 
for label in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label)
    if os.path.isdir(label_dir):
        for img_file in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_file)
            img = load_img(img_path, color_mode="grayscale")
            img = load_img(img_path, target_size=(256, 256), color_mode="grayscale")
            img_array = img_to_array(img) / 255.0  # Normalize
            images.append(img_array)
            labels.append(label)


images = np.array(images)
labels = np.array(labels)

class_names = ['normal', 'malignant', 'benign']

images = images.reshape(-1, 256, 256, 1)


label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(images, encoded_labels, test_size=0.2, random_state=42)


## Define the CNN Model

In [ ]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
    tf.keras.layers.BatchNormalization(),  
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),  
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),  
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 32)      320       
                                                                 
 batch_normalization_3 (Batc  (None, 254, 254, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 batch_normalization_4 (Batc  (None, 125, 125, 64)     256       
 hNormalization)                                                 
                                                      

## Apply Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation 
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)


datagen.fit(x_train)


In [ ]:
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=16),  
    validation_data=(x_test, y_test),
    epochs=30,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
)


Epoch 1/30
85/85 [==============================] - 38s 400ms/step - loss: 13.2049 - accuracy: 0.4212 - val_loss: 52.7434 - val_accuracy: 0.3531
Epoch 2/30
85/85 [==============================] - 31s 360ms/step - loss: 1.1098 - accuracy: 0.4718 - val_loss: 71.5525 - val_accuracy: 0.3531
Epoch 3/30
85/85 [==============================] - 30s 351ms/step - loss: 1.0472 - accuracy: 0.4198 - val_loss: 75.3885 - val_accuracy: 0.3531
Epoch 4/30
85/85 [==============================] - 31s 360ms/step - loss: 0.9885 - accuracy: 0.4421 - val_loss: 40.4122 - val_accuracy: 0.3531
Epoch 5/30
85/85 [==============================] - 31s 359ms/step - loss: 1.0391 - accuracy: 0.4421 - val_loss: 9.6734 - val_accuracy: 0.3976
Epoch 6/30
85/85 [==============================] - 29s 341ms/step - loss: 1.0216 - accuracy: 0.4287 - val_loss: 1.9194 - val_accuracy: 0.4510
Epoch 7/30
85/85 [==============================] - 30s 356ms/step - loss: 1.0016 - accuracy: 0.4718 - val_loss: 1.5292 - val_accuracy: 0

## Evaluate the Model

In [15]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=0)
print('\nTest accuracy:', test_acc)


Test accuracy: 0.6320474743843079


## Print Classification Report

In [ ]:
from sklearn.metrics import f1_score, classification_report
import numpy as np

# Make Predictions on the Test Set

y_pred_prob = model.predict(x_test)

# Convert probabilities to class predictions
y_pred = np.argmax(y_pred_prob, axis=1)


if len(y_test.shape) > 1 and y_test.shape[1] > 1:
    
    y_true = np.argmax(y_test, axis=1)
else:
    
    y_true = y_test

# Calculate F1 Score
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

# Print Results
print("Test Accuracy:", np.mean(y_true == y_pred))
print("F1 Score (Macro):", f1_macro)
print("F1 Score (Weighted):", f1_weighted)

# Print Detailed Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred))


11/11 [==============================] - 2s 163ms/step
Test Accuracy: 0.6320474777448071
F1 Score (Macro): 0.5254564705984668
F1 Score (Weighted): 0.5526790908625339

Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.92      0.66       118
           1       0.90      0.87      0.88       119
           2       0.17      0.02      0.04       100

    accuracy                           0.63       337
   macro avg       0.53      0.60      0.53       337
weighted avg       0.55      0.63      0.55       337



## Print 10-Fold Cross-Validation Results:

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import numpy as np

# Initialize KFold cross-validation with 10 splits
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Lists to store metrics for each fold
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []
true_positive_rates = []
false_positive_rates = []

# Loop over each fold
for train_index, val_index in kf.split(x_train):
    
    x_val_fold = x_train[val_index]
    y_val_fold = y_train[val_index]

    # Predict on the validation set
    y_val_pred_prob = model.predict(x_val_fold)
    y_val_pred = np.argmax(y_val_pred_prob, axis=1)

    # Calculate and store metrics
    accuracy = accuracy_score(y_val_fold, y_val_pred)
    precision = precision_score(y_val_fold, y_val_pred, average='macro')
    recall = recall_score(y_val_fold, y_val_pred, average='macro')
    f1 = f1_score(y_val_fold, y_val_pred, average='macro')


    if len(np.unique(y_val_fold)) == 2:
        roc_auc = roc_auc_score(y_val_fold, y_val_pred_prob[:, 1])
        roc_auc_scores.append(roc_auc)

    # Calculate TP and FP rates from confusion matrix
    cm = confusion_matrix(y_val_fold, y_val_pred)
    tp_rate = cm[1, 1] / (cm[1, 1] + cm[1, 0]) if (cm[1, 1] + cm[1, 0]) > 0 else 0
    fp_rate = cm[0, 1] / (cm[0, 1] + cm[0, 0]) if (cm[0, 1] + cm[0, 0]) > 0 else 0

    true_positive_rates.append(tp_rate)
    false_positive_rates.append(fp_rate)
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

# Calculate the average of each metric across all folds
print("10-Fold Cross-Validation Results:")
print("Average Accuracy:", np.mean(accuracy_scores))
print("Average Precision:", np.mean(precision_scores))
print("Average Recall:", np.mean(recall_scores))
print("Average F1 Score:", np.mean(f1_scores))
print("Average TP Rate:", np.mean(true_positive_rates))
print("Average FP Rate:", np.mean(false_positive_rates))

if roc_auc_scores:
    print("Average ROC AUC Score:", np.mean(roc_auc_scores))


5/5 [==============================] - 1s 127ms/step
10-Fold Cross-Validation Results:
Average Accuracy: 0.6166556108347154
Average Precision: 0.564604482902382
Average Recall: 0.6243303374506269
Average F1 Score: 0.5335183382516665
Average TP Rate: 0.9319428710550677
Average FP Rate: 0.023071854974294


# CNN With Augmentation and Learning Rate

## Define the Model

In [18]:
# Define an improved CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
    tf.keras.layers.BatchNormalization(),  # Batch normalization
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),  # Batch normalization
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),  # Batch normalization
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # Output layer for 3 classes
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 254, 254, 32)      320       
                                                                 
 batch_normalization_6 (Batc  (None, 254, 254, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 batch_normalization_7 (Batc  (None, 125, 125, 64)     256       
 hNormalization)                                                 
                                                      

## Apply Augmentation

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation for training
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

# Apply augmentation only to the training set
datagen.fit(x_train)


## Train the Model

In [20]:
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=16),  # Augmented training data
    validation_data=(x_test, y_test),
    epochs=30,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
)


Epoch 1/30
85/85 [==============================] - 32s 362ms/step - loss: 2.5249 - accuracy: 0.4606 - val_loss: 5.7473 - val_accuracy: 0.2967
Epoch 2/30
85/85 [==============================] - 32s 369ms/step - loss: 1.1436 - accuracy: 0.5163 - val_loss: 6.4885 - val_accuracy: 0.4896
Epoch 3/30
85/85 [==============================] - 32s 373ms/step - loss: 0.9740 - accuracy: 0.5104 - val_loss: 8.7301 - val_accuracy: 0.4629
Epoch 4/30
85/85 [==============================] - 31s 364ms/step - loss: 0.8972 - accuracy: 0.5319 - val_loss: 6.9374 - val_accuracy: 0.4451
Epoch 5/30
85/85 [==============================] - 30s 346ms/step - loss: 0.8473 - accuracy: 0.5594 - val_loss: 2.2573 - val_accuracy: 0.4095
Epoch 6/30
85/85 [==============================] - 30s 347ms/step - loss: 0.8008 - accuracy: 0.5840 - val_loss: 1.0079 - val_accuracy: 0.5549
Epoch 7/30
85/85 [==============================] - 30s 349ms/step - loss: 0.8207 - accuracy: 0.5914 - val_loss: 0.6747 - val_accuracy: 0.6677

## Evaluate The Model

In [22]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=0)
print('\nTest accuracy:', test_acc)


Test accuracy: 0.6676557660102844


## Print Classification Report


In [23]:
from sklearn.metrics import f1_score, classification_report
import numpy as np

# Make Predictions on the Test Set
# Get predicted labels (as probabilities)
y_pred_prob = model.predict(x_test)

# Convert probabilities to class predictions
y_pred = np.argmax(y_pred_prob, axis=1)

# Check if y_test is one-hot encoded or not
if len(y_test.shape) > 1 and y_test.shape[1] > 1:
    # y_test is one-hot encoded
    y_true = np.argmax(y_test, axis=1)
else:
    # y_test is not one-hot encoded
    y_true = y_test

# Calculate F1 Score
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

# Print Results
print("Test Accuracy:", np.mean(y_true == y_pred))
print("F1 Score (Macro):", f1_macro)
print("F1 Score (Weighted):", f1_weighted)

# Print Detailed Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred))


11/11 [==============================] - 1s 125ms/step
Test Accuracy: 0.6676557863501483
F1 Score (Macro): 0.6511998615657153
F1 Score (Weighted): 0.6543899013501677

Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.36      0.49       118
           1       0.83      0.89      0.86       119
           2       0.50      0.76      0.60       100

    accuracy                           0.67       337
   macro avg       0.69      0.67      0.65       337
weighted avg       0.70      0.67      0.65       337



## Print the 10-Fold Cross-Validation Results:

In [24]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import numpy as np

# Initialize KFold cross-validation with 10 splits
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Lists to store metrics for each fold
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []
true_positive_rates = []
false_positive_rates = []

# Loop over each fold
for train_index, val_index in kf.split(x_train):
   
    x_val_fold = x_train[val_index]
    y_val_fold = y_train[val_index]

    # Predict on the validation set
    y_val_pred_prob = model.predict(x_val_fold)
    y_val_pred = np.argmax(y_val_pred_prob, axis=1)

    # Calculate and store metrics
    accuracy = accuracy_score(y_val_fold, y_val_pred)
    precision = precision_score(y_val_fold, y_val_pred, average='macro')
    recall = recall_score(y_val_fold, y_val_pred, average='macro')
    f1 = f1_score(y_val_fold, y_val_pred, average='macro')

    # ROC AUC score can only be calculated if there are exactly two classes
    if len(np.unique(y_val_fold)) == 2:
        roc_auc = roc_auc_score(y_val_fold, y_val_pred_prob[:, 1])
        roc_auc_scores.append(roc_auc)

    # Calculate TP and FP rates from confusion matrix
    cm = confusion_matrix(y_val_fold, y_val_pred)
    tp_rate = cm[1, 1] / (cm[1, 1] + cm[1, 0]) if (cm[1, 1] + cm[1, 0]) > 0 else 0
    fp_rate = cm[0, 1] / (cm[0, 1] + cm[0, 0]) if (cm[0, 1] + cm[0, 0]) > 0 else 0

    true_positive_rates.append(tp_rate)
    false_positive_rates.append(fp_rate)
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

# Calculate the average of each metric across all folds
print("10-Fold Cross-Validation Results:")
print("Average Accuracy:", np.mean(accuracy_scores))
print("Average Precision:", np.mean(precision_scores))
print("Average Recall:", np.mean(recall_scores))
print("Average F1 Score:", np.mean(f1_scores))
print("Average TP Rate:", np.mean(true_positive_rates))
print("Average FP Rate:", np.mean(false_positive_rates))

if roc_auc_scores:
    print("Average ROC AUC Score:", np.mean(roc_auc_scores))


5/5 [==============================] - 1s 118ms/step
10-Fold Cross-Validation Results:
Average Accuracy: 0.6664842454394693
Average Precision: 0.6877979047891465
Average Recall: 0.6660144670673734
Average F1 Score: 0.6483521089885661
Average TP Rate: 0.9447864214185826
Average FP Rate: 0.14054147606779183
